# Titanic Dataset : Predicting Survivors

### Loading dataset

In [1]:
import pandas as pd
import numpy as np


In [2]:
train = pd.read_csv("./titanic/train.csv")
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Shape and Describe

In [3]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
train.shape

(891, 12)

## How to see the Number of people that survived ? ( Hint: value_counts()  )

In [5]:
train["Survived"].value_counts()

0    549
1    342
Name: Survived, dtype: int64

### Lets see the number of males and Females that survived

In [6]:
train["Survived"][train["Sex"] == 'male'].value_counts() # 1 : survived, 0: Died

0    468
1    109
Name: Survived, dtype: int64

In [7]:
train["Survived"][train["Sex"] == 'female'].value_counts() # 1 : survived, 0: Died

1    233
0     81
Name: Survived, dtype: int64

## Hmmm, ok but can you normalize these...sure

In [8]:
males = train["Survived"][train["Sex"] == 'male'].value_counts(normalize = True)
males

0    0.811092
1    0.188908
Name: Survived, dtype: float64

In [9]:
females = train["Survived"][train["Sex"] == 'female'].value_counts(normalize = True)
females

1    0.742038
0    0.257962
Name: Survived, dtype: float64

### So  the  rate  for  males  surviving   is  ~ 19 %  and   for  females  it  is  ~ 74%

## What About Age ? Remember : Women and Children first !

In [10]:
children = train["Survived"][train["Age"] < 18].value_counts(normalize = True)
children

1    0.539823
0    0.460177
Name: Survived, dtype: float64

### So about ~ 54 % children survived.

In [11]:
adults = train["Survived"][train["Age"] >= 18].value_counts(normalize = True)
adults

0    0.618968
1    0.381032
Name: Survived, dtype: float64

### And for the adults it was close to 62 %

# Lets get predicting ..... summon the tree

In [12]:
from sklearn import tree

### Hang on a minute .... we need to convert Sex and Class to integers and not to forget imputing all the missing values

In [13]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [14]:
# 1. C = Cherbourg ,  2. Q = Queenstown,   3.  S = Southampton
# Sex male = 0 , female = 1

In [15]:
train["Sex_int"] = 0
train["Sex_int"][train["Sex"] == "male"] = 0

train["Sex_int"][train["Sex"] == "female"] = 1

train[["Sex_int","Sex"]].head()

/home/shubham/Desktop/titanic/venv/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/shubham/Desktop/titanic/venv/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Sex_int,Sex
0,0,male
1,1,female
2,1,female
3,1,female
4,0,male


In [16]:
train["Embarked"] = train["Embarked"].fillna("S")

train["Embarked_int"] = 0
train["Embarked_int"][train["Embarked"] == 'C'] = 1
train["Embarked_int"][train["Embarked"] == 'Q'] = 2
train["Embarked_int"][train["Embarked"] == 'S'] = 3
train[["Embarked_int","Embarked"]].tail()

/home/shubham/Desktop/titanic/venv/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/shubham/Desktop/titanic/venv/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/shubham/Desktop/titanic/venv/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Embarked_int,Embarked
886,3,S
887,3,S
888,3,S
889,1,C
890,2,Q


## features

In [17]:
features_tree_one = train[["Pclass","Sex_int","Age","Embarked_int"]]

In [18]:
features_tree_one.values #  values converts pandas dataframe to array structre....perfect for inputs to train our tree

array([[  3.,   0.,  22.,   3.],
       [  1.,   1.,  38.,   1.],
       [  3.,   1.,  26.,   3.],
       ..., 
       [  3.,   1.,  nan,   3.],
       [  1.,   0.,  26.,   1.],
       [  3.,   0.,  32.,   2.]])

## Target

In [19]:
target = train["Survived"].values

## Training our first tree